In [1]:
import pandas as pd
import numpy as np

from keras.layers import Dense, LSTM
from keras.layers import Dropout
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
data = pd.read_excel(r'winning_moves.xlsx')
data = data[data['class']==True]
data.head()

,0,1,2,3,4,5,6,7,8,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [3]:
data['win_moves']= data.apply(lambda x: list(x[x=='x'].index), axis=1)

In [4]:
data.head()

,0,1,2,3,4,5,6,7,8,class,win_moves
0,x,x,x,x,o,o,x,o,o,True,"[0, 1, 2, 3, 6]"
1,x,x,x,x,o,o,o,x,o,True,"[0, 1, 2, 3, 7]"
2,x,x,x,x,o,o,o,o,x,True,"[0, 1, 2, 3, 8]"
3,x,x,x,x,o,o,o,b,b,True,"[0, 1, 2, 3]"
4,x,x,x,x,o,o,b,o,b,True,"[0, 1, 2, 3]"


In [20]:

class tictactoe:

    def __init__(self, inp, out, epochs, batchSize):
        self.epochs = epochs
        self.batchSize = batchSize
        self.numberOfInputs = inp
        self.numberOfOutputs = out
        self.model = Sequential()
        self.model.add(Dense(64, activation='relu', input_shape=(inp, )))
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(256, activation='relu'))
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(out, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    def train(self, dataset):
        input = []
        output = []
        for data in dataset:
            input.append(data[1])
            output.append(data[0])

        X = np.array(input).reshape((-1, self.numberOfInputs))
        y = to_categorical(output, num_classes=3)
        # Train and test data split
        boundary = int(0.8 * len(X))
        X_train = X[:boundary]
        X_test = X[boundary:]
        y_train = y[:boundary]
        y_test = y[boundary:]
        self.model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=self.epochs, batch_size=self.batchSize)

    def predict(self, data, index):
        return self.model.predict(np.array(data).reshape(-1, self.numberOfInputs))[0][index]

In [21]:
import os
import random
import copy

PLAYER_X = 'X'
PLAYER_O = 'O'
EMPTY = ' '
PLAYER_X_VAL = -1
PLAYER_O_VAL = 1
EMPTY_VAL = 0
HORIZONTAL_SEPARATOR = ' | '
VERTICAL_SEPARATOR = '---------------'
GAME_STATE_X = -1
GAME_STATE_O = 1
GAME_STATE_DRAW = 0
GAME_STATE_NOT_ENDED = 2


class Game:

    def __init__(self):
        self.resetBoard()
        self.trainingHistory = []

    def resetBoard(self):
        self.board = [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ]
        self.boardHistory = []

    def printBoard(self):
        print(VERTICAL_SEPARATOR)
        for i in range(len(self.board)):
            print(' ', end='')
            for j in range(len(self.board[i])):
                if PLAYER_X_VAL == self.board[i][j]:
                    print(PLAYER_X, end='')
                elif PLAYER_O_VAL == self.board[i][j]:
                    print(PLAYER_O, end='')
                elif EMPTY_VAL == self.board[i][j]:
                    print(EMPTY, end='')
                print(HORIZONTAL_SEPARATOR, end='')
            print(os.linesep)
            print(VERTICAL_SEPARATOR)

    def getGameResult(self):
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
                if self.board[i][j] == EMPTY_VAL:
                    return GAME_STATE_NOT_ENDED

        # Rows
        for i in range(len(self.board)):
            candidate = self.board[i][0]
            for j in range(len(self.board[i])):
                if candidate != self.board[i][j]:
                    candidate = 0
            if candidate != 0:
                return candidate

        # Columns
        for i in range(len(self.board)):
            candidate = self.board[0][i]
            for j in range(len(self.board[i])):
                if candidate != self.board[j][i]:
                    candidate = 0
            if candidate != 0:
                return candidate

        # First diagonal
        candidate = self.board[0][0]
        for i in range(len(self.board)):
            if candidate != self.board[i][i]:
                candidate = 0
        if candidate != 0:
            return candidate

        # Second diagonal
        candidate = self.board[0][2]
        for i in range(len(self.board)):
            if candidate != self.board[i][len(self.board[i]) - i - 1]:
                candidate = 0
        if candidate != 0:
            return candidate

        return GAME_STATE_DRAW


    def getAvailableMoves(self):
        availableMoves = []
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
                if (self.board[i][j]) == EMPTY_VAL:
                    availableMoves.append([i, j])
        return availableMoves

    def addToHistory(self, board):
        self.boardHistory.append(board)

    def printHistory(self):
        print(self.boardHistory)

    def move(self, position, player):
        availableMoves = self.getAvailableMoves()
        for i in range(len(availableMoves)):
            if position[0] == availableMoves[i][0] and position[1] == availableMoves[i][1]:
                self.board[position[0]][position[1]] = player
                self.addToHistory(copy.deepcopy(self.board))


    def simulate(self, playerToMove):
        while (self.getGameResult() == GAME_STATE_NOT_ENDED):
            availableMoves = self.getAvailableMoves()
            selectedMove = availableMoves[random.randrange(0, len(availableMoves))]
            self.move(selectedMove, playerToMove)
            if playerToMove == PLAYER_X_VAL:
                playerToMove = PLAYER_O_VAL
            else:
                playerToMove = PLAYER_X_VAL
        # Get the history and build the training set
        for historyItem in self.boardHistory:
            self.trainingHistory.append((self.getGameResult(), copy.deepcopy(historyItem)))

    def simulateNeuralNetwork(self, nnPlayer, model):
        playerToMove = PLAYER_X_VAL
        while (self.getGameResult() == GAME_STATE_NOT_ENDED):
            availableMoves = self.getAvailableMoves()
            if playerToMove == nnPlayer:
                maxValue = 0
                bestMove = availableMoves[0]
                for availableMove in availableMoves:
                    # get a copy of a board
                    boardCopy = copy.deepcopy(self.board)
                    boardCopy[availableMove[0]][availableMove[1]] = nnPlayer
                    if nnPlayer == PLAYER_X_VAL:
                        value = model.predict(boardCopy, 0)
                    else:
                        value = model.predict(boardCopy, 2)
                    if value > maxValue:
                        maxValue = value
                        bestMove = availableMove
                selectedMove = bestMove
            else:
                selectedMove = availableMoves[random.randrange(0, len(availableMoves))]
            self.move(selectedMove, playerToMove)
            if playerToMove == PLAYER_X_VAL:
                playerToMove = PLAYER_O_VAL
            else:
                playerToMove = PLAYER_X_VAL

    def getTrainingHistory(self):
        return self.trainingHistory

    def simulateManyGames(self, playerToMove, numberOfGames):
        playerXWins = 0
        playerOWins = 0
        draws = 0
        for i in range(numberOfGames):
            self.resetBoard()
            self.simulate(playerToMove)
            if self.getGameResult() == PLAYER_X_VAL:
                playerXWins = playerXWins + 1
            elif self.getGameResult() == PLAYER_O_VAL:
                playerOWins = playerOWins + 1
            else: draws = draws + 1
        totalWins = playerXWins + playerOWins + draws
        print ('X Wins: ' + str(int(playerXWins * 100/totalWins)) + '%')
        print('O Wins: ' + str(int(playerOWins * 100 / totalWins)) + '%')
        print('Draws: ' + str(int(draws * 100 / totalWins)) + '%')


    def simulateManyNeuralNetworkGames(self, nnPlayer, numberOfGames, model):
        nnPlayerWins = 0
        randomPlayerWins = 0
        draws = 0
        print ("NN player")
        print (nnPlayer)
        for i in range(numberOfGames):
            self.resetBoard()
            self.simulateNeuralNetwork(nnPlayer, model)
            if self.getGameResult() == nnPlayer:
                nnPlayerWins = nnPlayerWins + 1
            elif self.getGameResult() == GAME_STATE_DRAW:
                draws = draws + 1
            else: randomPlayerWins = randomPlayerWins + 1
        totalWins = nnPlayerWins + randomPlayerWins + draws
        print ('X Wins: ' + str(int(nnPlayerWins * 100/totalWins)) + '%')
        print('O Wins: ' + str(int(randomPlayerWins * 100 / totalWins)) + '%')
        print('Draws: ' + str(int(draws * 100 / totalWins)) + '%')


In [22]:
game = Game()

game.simulateManyGames(1, 100)

ticTacToeModel = tictactoe(9, 3, 100, 32)
ticTacToeModel.train(game.getTrainingHistory())

X Wins: 24%
O Wins: 63%
Draws: 13%
Epoch 1/100
23/23 [==============================] - 1s 8ms/step - loss: 0.8591 - accuracy: 0.6333 - val_loss: 0.8975 - val_accuracy: 0.6056
Epoch 2/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7070 - accuracy: 0.7000 - val_loss: 0.8574 - val_accuracy: 0.6667
Epoch 3/100
23/23 [==============================] - 0s 3ms/step - loss: 0.5941 - accuracy: 0.7347 - val_loss: 0.9087 - val_accuracy: 0.6944
Epoch 4/100
23/23 [==============================] - 0s 3ms/step - loss: 0.5095 - accuracy: 0.7889 - val_loss: 0.9504 - val_accuracy: 0.6944
Epoch 5/100
23/23 [==============================] - 0s 3ms/step - loss: 0.4321 - accuracy: 0.8250 - val_loss: 1.1337 - val_accuracy: 0.6889
Epoch 6/100
23/23 [==============================] - 0s 3ms/step - loss: 0.3987 - accuracy: 0.8333 - val_loss: 1.1579 - val_accuracy: 0.6000
Epoch 7/100
23/23 [==============================] - 0s 3ms/step - loss: 0.3617 - accuracy: 0.8500 - val_loss: 1.1284 -

23/23 [==============================] - 0s 3ms/step - loss: 0.1461 - accuracy: 0.9208 - val_loss: 3.1282 - val_accuracy: 0.6389
Epoch 59/100
23/23 [==============================] - 0s 3ms/step - loss: 0.1471 - accuracy: 0.9222 - val_loss: 3.3150 - val_accuracy: 0.6056
Epoch 60/100
23/23 [==============================] - 0s 3ms/step - loss: 0.1504 - accuracy: 0.9167 - val_loss: 3.3146 - val_accuracy: 0.6444
Epoch 61/100
23/23 [==============================] - 0s 3ms/step - loss: 0.1465 - accuracy: 0.9194 - val_loss: 3.4035 - val_accuracy: 0.6556
Epoch 62/100
23/23 [==============================] - 0s 3ms/step - loss: 0.1477 - accuracy: 0.9167 - val_loss: 3.2833 - val_accuracy: 0.6444
Epoch 63/100
23/23 [==============================] - 0s 3ms/step - loss: 0.1437 - accuracy: 0.9153 - val_loss: 3.3814 - val_accuracy: 0.6389
Epoch 64/100
23/23 [==============================] - 0s 3ms/step - loss: 0.1465 - accuracy: 0.9125 - val_loss: 3.4146 - val_accuracy: 0.6722
Epoch 65/100
23/23 

In [25]:
print ("Simulating with Neural Network as X Player:")
game.simulateManyNeuralNetworkGames(PLAYER_X_VAL, 100, ticTacToeModel)


Simulating with Neural Network as X Player:
NN player
-1
X Wins: 72%
O Wins: 1%
Draws: 27%
